In [ ]:
import os
from pathlib import Path

import pandas as pd
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import matplotlib.pyplot as plt

In [ ]:
event_file = '/home/tolhsadum/events.out.tfevents.1691579244.pea2.dartmoor.3370.1'

# load the file
event_acc = EventAccumulator(event_file)
# print scalars
event_acc.Reload()
scalar_tags = event_acc.Tags()['scalars']

# Print the scalar tag names
print("Scalar Tags:")
for tag in scalar_tags:
    print(tag)
    
filtered_tags = ['epoch_train_loss', 'epoch_train_ctr_loss', 'val_mean_dice', 'val_distance', 'ctr_val_volume']

# do the same for all the tags containing 'best'
best_tags = [tag for tag in scalar_tags if 'best' in tag]
"""
epoch_train_loss
epoch_train_ctr_loss
val_mean_dice
ctr_val_loss
ctr_val_volume
val_distance
val_best_mean_dice
val_best_mean_loss
val_best_mean_distance

train = training
ctr = controls
val = validation
"""
translate_dict = {
    'epoch_train_loss': 'Training Loss',
    'epoch_train_ctr_loss': 'Training Controls Loss',
    'val_mean_dice': 'Validation Mean Dice',
    'ctr_val_loss': 'Validation Controls Loss',
    'ctr_val_volume': 'Validation Controls Volume',
    'val_distance': 'Validation Distance',
    'val_best_mean_dice': 'Validation Best Mean Dice',
    'val_best_mean_loss': 'Validation Best Mean Loss',
    'val_best_mean_distance': 'Validation Best Mean Distance'
}

In [ ]:
# plot each scalar in a different subplot
fig, axs = plt.subplots(len(filtered_tags), 1, figsize=(10, 10))
for i, tag in enumerate(filtered_tags):
    x, y = zip(*[(s.step, s.value) for s in event_acc.Scalars(tag)])
    axs[i].plot(x, y)
    axs[i].set_title(tag)
    axs[i].set_xlabel('Epoch')
    axs[i].set_ylabel(tag)
plt.tight_layout()
plt.show()
# plot each scalar in a different subplot
fig, axs = plt.subplots(len(best_tags), 1, figsize=(10, 10))
for i, tag in enumerate(best_tags):
    x, y = zip(*[(s.step, s.value) for s in event_acc.Scalars(tag)])
    axs[i].plot(x, y)
    axs[i].set_title(tag)
    axs[i].set_xlabel('Epoch')
    axs[i].set_ylabel(tag)
plt.tight_layout()
plt.show()



In [ ]:
# do the same for both tags lists but point to the best value in the plot and display its value
# each tag containing "loss", "distance" or "volume" the best value is the lowest
# create a list like the tags lists but with the best value function to apply
best_func_filtered_tags = [min if 'loss' in tag or 'distance' in tag or 'volume' in tag else max for tag in filtered_tags]
best_func_best_tags = [min if 'loss' in tag or 'distance' in tag or 'volume' in tag else max for tag in best_tags]

# plot each scalar in a different subplot
fig, axs = plt.subplots(len(filtered_tags), 1, figsize=(10, len(filtered_tags)*5))
for i, tag in enumerate(filtered_tags):
    x, y = zip(*[(s.step, s.value) for s in event_acc.Scalars(tag)])
    axs[i].plot(x, y)
    axs[i].set_xlabel('Epoch')
    axs[i].set_ylabel(tag)
    # find the best value
    best_value_func = best_func_filtered_tags[i]
    best_value = best_value_func(y)
    best_epoch = x[y.index(best_value)]
    axs[i].plot(best_epoch, best_value, 'ro')
    axs[i].set_title(f'{tag} (Best value: {best_value:.4f})')
plt.tight_layout()
plt.show()

# plot each scalar in a different subplot
fig, axs = plt.subplots(len(best_tags), 1, figsize=(10, len(best_tags)*5))
for i, tag in enumerate(best_tags):
    x, y = zip(*[(s.step, s.value) for s in event_acc.Scalars(tag)])
    axs[i].plot(x, y)
    axs[i].set_xlabel('Epoch')
    axs[i].set_ylabel(tag)
    # find the best value
    best_value_func = best_func_best_tags[i]
    best_value = best_value_func(y)
    best_epoch = x[y.index(best_value)]
    axs[i].plot(best_epoch, best_value, 'ro')
    axs[i].set_title(f'{tag} (Best value: {best_value:.4f})')
plt.tight_layout()
plt.show()

In [ ]:
# plot each scalar in a different subplot and save each plot as a separate image file
output_folder = Path('/data/Dropbox (GIN)/result_ucl_lesseg')  # replace with your output folder path
for i, tag in enumerate(filtered_tags):
    fig, ax = plt.subplots(figsize=(10, 5))
    x, y = zip(*[(s.step, s.value) for s in event_acc.Scalars(tag)])
    ax.plot(x, y)
    ax.set_xlabel('Epoch')
    ax.set_ylabel(tag)
    # find the best value
    best_value_func = best_func_filtered_tags[i]
    best_value = best_value_func(y)
    best_epoch = x[y.index(best_value)]
    ax.plot(best_epoch, best_value, 'ro')
    ax.set_title(f'Best value: {best_value:.4f}')
    plt.tight_layout()
    plt.savefig(output_folder / f'{tag}.png')
    plt.close(fig)  # close the figure

# do the same for the best_tags list
for i, tag in enumerate(best_tags):
    fig, ax = plt.subplots(figsize=(10, 5))
    x, y = zip(*[(s.step, s.value) for s in event_acc.Scalars(tag)])
    ax.plot(x, y)
    ax.set_xlabel('Epoch')
    ax.set_ylabel(tag)
    # find the best value
    best_value_func = best_func_best_tags[i]
    best_value = best_value_func(y)
    best_epoch = x[y.index(best_value)]
    ax.plot(best_epoch, best_value, 'ro')
    ax.set_title(f'Best value: {best_value:.4f}')
    plt.tight_layout()
    plt.savefig(output_folder / f'{tag}.png')
    plt.close(fig)  # close the figure

In [ ]:
def load_tensorboard_file(event_file, verbose=False):
    # Load the TensorBoard file
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()

    # If verbose is True, print all the tags from the file
    if verbose:
        print("Tags:")
        for tag in event_acc.Tags()['scalars']:
            print(tag)

    return event_acc

def create_and_save_plots(event_acc, best_func_dict=None, output_folder=None, display_plot=True):
    # Get the list of scalar tags
    scalar_tags = event_acc.Tags()['scalars']

    # If best_func_dict is not provided, use a default function (min) for all tags
    if best_func_dict is None:
        best_func_dict = {tag: min for tag in scalar_tags}

    # Create and save a plot for each tag
    for tag in scalar_tags:
        fig, ax = plt.subplots(figsize=(10, 5))
        x, y = zip(*[(s.step, s.value) for s in event_acc.Scalars(tag)])
        ax.plot(x, y)
        ax.set_xlabel('Epoch')
        ax.set_ylabel(tag)
        # Find the best value
        best_value_func = best_func_dict.get(tag, min)  # Use min if the tag is not in best_func_dict
        best_value = best_value_func(y)
        best_epoch = x[y.index(best_value)]
        ax.plot(best_epoch, best_value, 'ro')
        ax.set_title(f'Best value: {best_value:.4f}')
        plt.tight_layout()
        # Save the plot if output_folder is provided
        if output_folder is not None:
            plt.savefig(output_folder / f'{tag}.png')
        # Display the plot if display_plot is True
        if display_plot:
            plt.show()
        plt.close(fig)  # Close the figure

